In [1]:
import numpy as np
import pandas as pd
import cv2
import PIL
import pytesseract

In [2]:
img_cv = cv2.imread('images/000.jpg')

cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
text_cv = pytesseract.image_to_string(img_cv)

In [4]:
print(text_cv)

CORPORATE PACKAGING & PROMOTIONS

Bill of Lading

To
Consignee — SAM'S DIST.CTR #6493

Street 1000 S. CUCAMONGA AVE

     

BILL OF LADING NUMBER: 8175105
PO#
BILL DATE:
FROM

AR50 CORPORATE PACKAGING

   

Shipper

         

 

Destination

   

     

 

City/State/Zip ONTARIO, CA 91761

   
 
  

 

THIRD PARTY BILLING, SEND TO

Name Arizona Beverages LLC
C

ompany Arizona Beverages LLC
Street

City/State/Zip

Street 1555 S. ARCHIBALD AVENUE
Origin
City/State/Zip ONTARIO, CA 91761

 
      

 

Special Instructions: —T
~~ SHIPPER'S INSTRUCTIONS \

 
  
 

 

NO.
SHIPPING
CASES

NO.
SHIPPING
PALLETS

DESCRIPTION OF ARTICLES |! !
SPECIAL MARKS & EXCEPTIONS: ) .-y)).::

 

 

UNNI

Equip 'D: 317 7
Equip Arivat: 12/20/19 0459
Carrier: BAWB °
Seal: 3289785

REMIT C.0.D.

ADDRESS:
Reseal:
Door/Zone:
Del Date:

APPOINTMENT
12/20/19 0500

' have read and understan
Appointment / Drop Rules

NOTE: W here the rate is depende
on value, shippers are required to
State specifically in writing the

In [5]:
data = pytesseract.image_to_data(img_cv)

In [6]:
#data.split('\n')
dataList = list(map(lambda x: x.split('\t'),data.split('\n')))
df = pd.DataFrame(dataList[1:],columns=dataList[0])

In [7]:
df.head(1000)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,1653,2339,-1,
1,2,1,1,0,0,0,188,234,541,27,-1,
2,3,1,1,1,0,0,188,234,541,27,-1,
3,4,1,1,1,1,0,188,234,541,27,-1,
4,5,1,1,1,1,1,188,234,160,27,96,CORPORATE
...,...,...,...,...,...,...,...,...,...,...,...,...
551,5,1,52,1,7,4,1323,1790,25,17,96,Bill
552,5,1,52,1,7,5,1357,1792,16,16,96,of
553,5,1,52,1,7,6,1381,1793,63,23,92,Lading
554,5,1,52,1,7,7,1454,1797,54,17,96,terms


In [8]:
df.dropna(inplace=True) # drop the missing in rows
col_int = ['level','page_num','block_num','par_num','line_num','word_num','left','top','width','height','conf']
df[col_int] = df[col_int].astype(int)

In [9]:
df.dtypes

level         int64
page_num      int64
block_num     int64
par_num       int64
line_num      int64
word_num      int64
left          int64
top           int64
width         int64
height        int64
conf          int64
text         object
dtype: object

In [10]:
image = img_cv.copy()
level = 'word'
for l,x,y,w,h,c,txt in df[['level','left','top','width','height','conf','text']].values:
    #print(l,x,y,w,h,c)
    if level == 'page':
        if l == 1:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,0),2)
        else:
            continue
            
    elif level == 'block':
        if l == 2:
            cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
        else:
            continue
    
    elif level == 'para':
        if l == 3:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
        else:
            continue
    
    elif level == 'line':
        if l == 4:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),2)
        else:
            continue
            
    elif level == 'word':
        if l == 5:
            cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(image,txt,(x,y),cv2.FONT_HERSHEY_PLAIN,1,(255,0,0),2)
        else:
            continue
            
cv2.imshow("",image)
cv2.waitKey()
cv2.destroyAllWindows()